In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

from sklearn.preprocessing import MultiLabelBinarizer

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange
import math

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from scipy.sparse.linalg import svds

from sklearn.utils import shuffle

import gc

from sklearn.preprocessing import LabelEncoder
from scipy import sparse

from sklearn.metrics.pairwise import cosine_similarity

from numpy import linalg as LA

import statistics

Mounted at /content/gdrive


### 1. Prepare data



#### 1.1 rs_cf_watch_history.csv

In [23]:
rs_cf = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/rs_cf/rs_cf_watch_history.csv', sep=',', error_bad_lines=False)
rs_cf

b'Skipping line 3654: expected 6 fields, saw 7\nSkipping line 4675: expected 6 fields, saw 7\nSkipping line 5527: expected 6 fields, saw 7\nSkipping line 6099: expected 6 fields, saw 7\nSkipping line 7718: expected 6 fields, saw 7\nSkipping line 8692: expected 6 fields, saw 7\nSkipping line 9152: expected 6 fields, saw 7\nSkipping line 9366: expected 6 fields, saw 7\nSkipping line 11334: expected 6 fields, saw 7\nSkipping line 12337: expected 6 fields, saw 7\nSkipping line 13099: expected 6 fields, saw 7\nSkipping line 13934: expected 6 fields, saw 7\nSkipping line 14107: expected 6 fields, saw 7\nSkipping line 15164: expected 6 fields, saw 7\nSkipping line 15530: expected 6 fields, saw 7\nSkipping line 15759: expected 6 fields, saw 7\nSkipping line 17186: expected 6 fields, saw 7\nSkipping line 18940: expected 6 fields, saw 7\nSkipping line 20086: expected 6 fields, saw 7\nSkipping line 22571: expected 6 fields, saw 7\nSkipping line 22919: expected 6 fields, saw 7\nSkipping line 23558

,member_id,content_id,content_name,duration,date_time,filename_date
0,7977837,135643,Tân Bảng Phong Thần - Phần 2,2770,2021-06-01 00:00:05,20210601
1,3905329,135789,Thiên La Địa Võng,5378,2021-05-31 23:57:51,20210601
2,2969037,136263,Đưa Con Về Nhà,48,2021-05-31 23:57:54,20210601
3,7673685,130123,Mãng Xà,195,2021-06-01 00:10:14,20210601
4,12029103,132759,Con Rối Hào Môn - Tập 3,2574,2021-06-01 00:08:09,20210601
...,...,...,...,...,...,...
5038264,13143187,136424,Nữ Sát Thủ,8288,2021-06-30 05:17:46,20210630
5038265,659253,136484,Mộ Bạch Thủ,2406,2021-06-30 12:52:15,20210630
5038266,6619599,125059,Đông Cung,36,2021-06-30 12:54:33,20210630
5038267,3967103,136260,Đội Đặc Nhiệm CID,1788,2021-06-30 12:52:39,20210630


#### Preprocessing



```
# Process tạo ra trường dữ liệu mới làm kết quả dự đoán.
```



In [24]:
dr_data = rs_cf.copy()

#Sum duration and count view time
view_count = dr_data.groupby(by=['member_id','content_id'], as_index=False).size()
dr_data = dr_data.groupby(by=['member_id','content_id'], as_index=False).sum()
dr_data['view_time'] = view_count['size']

del(view_count)
gc.collect()

176

In [25]:
#Create enjoyment column:
def enjoyment_calculate(median, view_time, duration):
  if view_time > 1 and duration >= median:
    return 3
  elif view_time == 1 and duration >= median:
    return 2
  elif view_time > 1 and duration < median:
    return 2
  elif view_time == 1 and duration < median:
    return 1

d_median = statistics.median(dr_data.duration)
dr_data['enjoyment'] = [enjoyment_calculate(d_median, dr_data.view_time[x], dr_data.duration[x]) for x in range(len(dr_data))]

In [26]:
#drop meanless column:
dr_data = dr_data.drop([ 'filename_date'], axis = 1)

#encode member_id, content_id:
le1 = LabelEncoder()
dr_data['encode_member_id'] = le1.fit_transform(dr_data.member_id)


In [27]:
dr_data

,member_id,content_id,duration,view_time,enjoyment,encode_member_id
0,3747,128141,69,4,2,0
1,3747,131909,4177,2,3,0
2,3747,135739,51,1,1,0
3,3747,135743,3,1,1,0
4,3747,136128,41,1,1,0
...,...,...,...,...,...,...
1396286,14641535,136456,3,1,1,317361
1396287,14641673,130107,29,1,1,317362
1396288,14641673,132535,10,1,1,317362
1396289,14641673,132731,48,1,1,317362


### 2. Các hàm bổ trợ


In [ ]:
#similarity matrix :
def similarity_matrix(dr_data, utility, field):
  ids = dr_data[field].unique()
  id_1 = []
  id_2=[]
  sim = []
  
  for i in tqdm(range(0, len(ids) -1)):
    vector_1 = utility.getrow(i).toarray()

    for x in range(i+1, len(ids)):
      vector_2 = utility.getrow(ids[x]).toarray()
      simi_score = cosine_similarity(vector_1,vector_2)
      id_1.append(ids[i])
      id_2.append(ids[x])
      sim.append(simi_score[0][0])

  s_matrix = pd.DataFrame({'id_1':id_1,'id_2':id_2,'similarity':sim})
  return s_matrix

### 3. User_user_filtering


#### 3.1 Chia tập train và test

In [ ]:
#get sample users:
sample_users = np.arange(3000)
dr_data = dr_data.loc[dr_data.encode_member_id.isin(sample_users)]


NameError: ignored

In [ ]:
#Encdoe content_id
dr_data['encode_content_id'] = le.fit_transform(dr_data.content_id)

In [ ]:
train, test = train_test_split(dr_data, test_size = 0.2)

test = test.reset_index(drop = True)

#### 3.2 Tạo Utility matrix

In [ ]:
np_dr_data = np.array(dr_data)

utility_maxtrix = sparse.csr_matrix((np_dr_data[:,4], (np_dr_data[:,5], np_dr_data[:,6])), shape = (np.max(np_dr_data[:,5]+1), np.max(np_dr_data[:,6]+1)), dtype = np.float)

In [ ]:
utility_maxtrix

#### 3.3 Thực hiện và tính toán

In [ ]:
def user_user_func(similarity_matrix, utility_maxtrix, train,user_id, content_id, k_neighbors):

  e_u_id = train.loc[train.member_id == user_id].reset_index(drop = True)#lấy encode member id
  if len(e_u_id) == 0:
    return 0 
  e_u_id = e_u_id.encode_member_id[0]

  e_c_id = train.loc[train.content_id == content_id].reset_index(drop = True)#lấy encode content id
  if len(e_c_id) == 0:
    return 0
  e_c_id = e_c_id.encode_content_id[0]
  
  u_s_mtx = similarity_matrix.loc[(similarity_matrix.id_1 == e_u_id) | (similarity_matrix.id_2 == e_u_id)].sort_values(by = ['similarity'], ascending = False).reset_index(drop=True)#lấy similarity và ranking

  sum = 0
  sum_para = 0
  k = 0
  for i in range(len(u_s_mtx)):#len(u_s_mtx): đúng bằng số user-1
    n_w_hs = utility_maxtrix.getrow(i).toarray()[0]
    if n_w_hs[e_c_id] != 0:
      sum += abs(u_s_mtx.similarity[i])
      sum_para += u_s_mtx.similarity[i]*n_w_hs[e_c_id]
      k += 1
    if k == k_neighbors:
      break
  
  if sum != 0:
    return sum_para/sum
  else:
    return 0

In [ ]:
y_pred = []
y_true = []

s_matrix = similarity_matrix(train, utility_maxtrix, field='encode_member_id')

for x in tqdm(range(len(test))):
  pred = user_user_func(s_matrix, utility_maxtrix, train,test.member_id[x], test.content_id[x], k_neighbors = 10)
  if pred != 0:
    y_pred.append(pred)
    y_true.append(test.enjoyment[x])

In [ ]:
RMSE = math.sqrt(mean_squared_error(y_true, y_pred))
RMSE

#### 3.4 Tạo mô hình và đưa ra kết quả

In [ ]:
s_matrix = similarity_matrix(dr_data, utility_maxtrix, field='encode_member_id') 

In [ ]:
def n_w_content(user_id, dr_data):
  w_hs = dr_data.loc[dr_data.member_id==user_id].content_id.tolist()
  c_list = [x for x in dr_data.content_id.unique() if x not in w_hs]

  return c_list

def choose_id(a, b, c):
  if a != c:
    return a
  else:
    return b

def user_user_recommend(similarity_matrix, utility_matrix, dr_data, user_id, k_neighbor):
  e_u_id = train.loc[train.member_id == user_id].reset_index(drop = True)#lấy encode member id
  if len(e_u_id) == 0:
    return 0 
  e_u_id = e_u_id.encode_member_id[0]

  #get similarity user_id:
  u_s_mtx = similarity_matrix.loc[(similarity_matrix.id_1 == e_u_id) | (similarity_matrix.id_2 == e_u_id)].sort_values(by = ['similarity'], ascending = False).reset_index(drop=True)#lấy similarity và ranking

  #list of content:
  c_list = n_w_content(user_id, dr_data)

  pred = []

  for content in tqdm(c_list):
    e_c_id = dr_data.loc[(dr_data.content_id == content) ].reset_index(drop = True).encode_content_id[0]#lấy encode content id

    sum = 0
    sum_para = 0
    k = 0

    for i in range(len(u_s_mtx)):
      e_nu_id = choose_id(u_s_mtx.id_1[i], u_s_mtx.id_2[i], e_u_id)
      n_w_hs = utility_maxtrix.getrow(e_nu_id).toarray()[0]
      if n_w_hs[e_c_id] != 0:
        sum += abs(u_s_mtx.similarity[i])
        sum_para += u_s_mtx.similarity[i]*n_w_hs[e_c_id]
        k += 1
      if k == k_neighbor:
        break
    
    if sum != 0:
      pred.append(sum_para/sum)
    else:
      pred.append(0)

  recommend_ranking = pd.DataFrame({'content_id':c_list, 'prediction': pred}).sort_values(by = ['prediction'], ascending = False).reset_index(drop = True)
  return recommend_ranking

In [ ]:

recommend = user_user_recommend(s_matrix, utility_maxtrix, dr_data, dr_data.member_id[200], 10)

In [ ]:
print('User_id:', dr_data.member_id[200])
recommend

### 4. Item_item Filtering

#### 4.1 Chia tập train và test

In [ ]:
#encode content_id
le = LabelEncoder()
dr_data['encode_content_id'] = le.fit_transform(dr_data.content_id)

In [ ]:
train, test = train_test_split(dr_data, test_size = 0.2)

train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [ ]:
train

,member_id,content_id,duration,view_time,enjoyment,encode_member_id,encode_content_id
0,12910009,135279,66,1,1,238589,3020
1,2802147,136109,58,1,1,67340,3242
2,963333,124003,162477,79,3,23176,976
3,87495,126679,19708,58,3,1184,1531
4,2245689,135861,362,1,2,55792,3130
...,...,...,...,...,...,...,...
1117027,7382175,135911,200,1,1,169143,3155
1117028,11839797,135815,4052,5,3,210729,3108
1117029,12976539,133435,14883,11,3,241179,2964
1117030,6313537,136338,5407,1,2,144549,3349


#### 4.2 Tạo utility matrix

In [ ]:
np_dr_data = np.array(train)

utility_maxtrix = sparse.csr_matrix((np_dr_data[:,4], (np_dr_data[:,6], np_dr_data[:,5])), shape = (np.max(np_dr_data[:,6]+1), np.max(np_dr_data[:,5]+1)), dtype = np.float)

In [ ]:
utility_maxtrix

<3478x317364 sparse matrix of type '<class 'numpy.float64'>'
	with 1117032 stored elements in Compressed Sparse Row format>

#### 4.3 Thực nghiệm

In [ ]:
i_s_matrix = similarity_matrix(train, utility_maxtrix, field='encode_content_id')
i_s_matrix['simi_multi'] = [x*1000000 for x in i_s_matrix.similarity]

i_s_matrix.to_csv('/content/gdrive/MyDrive/Đại học/Data/rs_cf/i_s_mtx.csv')



```
# Sau khi lưu vào file csv có sai số tuy nhiên sai số nhỏ 1x10^-17 nên không ảnh hưởng nhiều đến kết quả.
```



In [ ]:
i_s_matrix = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/rs_cf/i_s_mtx.csv')
i_s_matrix['similarity'] = [x/1000000 for x in i_s_matrix.simi_multi]


In [ ]:
def item_item_func(similarity_matrix, utility_maxtrix, train,user_id, content_id):

  e_u_id = train.loc[train.member_id == user_id].reset_index(drop = True)#lấy encode member id
  if len(e_u_id) == 0:
    print('1')
    return 0 
  e_u_id = e_u_id.encode_member_id[0]

  e_c_id = train.loc[train.content_id == content_id].reset_index(drop = True)#lấy encode content id
  if len(e_c_id) == 0:
    print('2')
    return 0
  e_c_id = e_c_id.encode_content_id[0]
  
  c_s_mtx = similarity_matrix.loc[(similarity_matrix.id_1 == e_c_id) | (similarity_matrix.id_2 == e_c_id)].sort_values(by = ['similarity'], ascending = False).reset_index(drop=True)#lấy similarity và ranking
  id_1 = np.array(c_s_mtx.id_1)
  id_2 = np.array(c_s_mtx.id_2)
  c_ids = train.encode_content_id.unique()

  sum = 0
  sum_para = 0
  k = 0
  
  n_w_hs = n_w_hs = utility_maxtrix.getrow(e_u_id).toarray()[0]

  for x in c_ids:
    if (x in id_1 or x in id_2) and x != e_c_id: 
      
      if n_w_hs[x] != 0:
        simi_score = c_s_mtx.loc[(c_s_mtx.id_1 == x) | (c_s_mtx.id_2 == x)].reset_index(drop=True)
        sum += abs(simi_score.similarity[0])
        sum_para += simi_score.similarity[0]*n_w_hs[x]

  if sum != 0:
    return sum_para/sum
  else:
    return 0

In [ ]:
utility = utility_maxtrix.transpose()

y_pred = []
y_true = []

for x in tqdm(range(len(test))):
  pred = item_item_func(i_s_matrix, utility, train,test.member_id[x], test.content_id[x])
  if pred != 0:
    y_pred.append(pred)
    y_true.append(test.enjoyment[x])

In [ ]:
RMSE = math.sqrt(mean_squared_error(y_true, y_pred))
RMSE

0.9311756000141276

#### 4.4 Item_item_recommend

In [ ]:
i_s_matrix = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/rs_cf/i_s_mtx.csv')
i_s_matrix['similarity'] = [x/1000000 for x in i_s_matrix.simi_multi]


In [ ]:
def item_item_rec(s_matrix, utility_maxtrix, dr_data, user_id):
  c_list = n_w_content(user_id, dr_data)
  
  pred = []
  for x in tqdm(range(len(c_list))):
    pred.append(item_item_func(s_matrix, utility_maxtrix, dr_data,user_id, c_list[x]))

  recommend_ranking = pd.DataFrame({'content_id':c_list, 'prediction': pred}).sort_values(by = ['prediction'], ascending = False).reset_index(drop = True)
  return recommend_ranking
  

In [ ]:
utility = utility_maxtrix.transpose()
user_id =  dr_data.member_id[5]
print('User_id: ', user_id)
item_item_rec(i_s_matrix, utility, dr_data, user_id)


User_id:  3747


,content_id,prediction
0,130397,3.0
1,128429,3.0
2,128575,3.0
3,132307,3.0
4,136326,3.0
...,...,...
3466,128625,0.0
3467,130199,0.0
3468,130419,0.0
3469,135823,0.0


### 5. Matrix factorziation


#### 5.1 Preprocessing 

In [31]:
#encode content_id
le = LabelEncoder()
dr_data['encode_content_id'] = le.fit_transform(dr_data.content_id)
dr_data

,member_id,content_id,duration,view_time,enjoyment,encode_member_id,encode_content_id
0,3747,128141,69,4,2,0,1808
1,3747,131909,4177,2,3,0,2445
2,3747,135739,51,1,1,0,3070
3,3747,135743,3,1,1,0,3072
4,3747,136128,41,1,1,0,3259
...,...,...,...,...,...,...,...
1396286,14641535,136456,3,1,1,317361,3430
1396287,14641673,130107,29,1,1,317362,2266
1396288,14641673,132535,10,1,1,317362,2638
1396289,14641673,132731,48,1,1,317362,2705


#### 5.1 Process data class

In [32]:
class Data(object):
  def __init__(self, dataset, split_rate):
    self.dataset = dataset
    self.split_rate = split_rate
    self.train, self.test = self.split_train_test(self.dataset)

    #do userid, item_id -=1 ở module spli_train_test
    self.n_users = np.max(self.train[:,5] + 1)
    self.n_items = np.max(self.train[:,6] + 1)

    self.Ytrain = self.utility_matrix(self.train)
    self.Ytest = self.utility_matrix(self.test)

    self.n_ratings = self.train.shape[0]# đếm số rating

    self.p_train_ratings = self.train[:,4:]
    self.p_test_ratings = self.test[:,4:]

  #tach tập train và test
  def split_train_test(self, dataset):
    train, test = train_test_split(dataset, test_size = self.split_rate, shuffle = True)
    train = np.array(train)
    test = np.array(test)
    
    return train, test

  #Tạo matran thua
  def utility_matrix(self, data_mtx):
    Y = np.zeros(shape = (self.n_items, self.n_users))

    #Matran thua y luu rating của người dùng
    Y = sparse.csr_matrix((data_mtx[:,4], (data_mtx[:,6], data_mtx[:,5])), shape = (self.n_items, self.n_users), dtype = np.float)
    
    return Y
    

In [33]:
dataset = Data(dataset = dr_data, split_rate=0.2)

#### 5.2 Class model
Bao gồm các func như: 
- Khởi tạo U,I
- Update U,I
- Dự đoán
- Tính toán hàm loss
- Tính RMSE


- Do việc lưu ma trận thưa và dùng phép tính lên ma trận thưa là gây tràn RAM nên cần sử dụng công thức tính toán riêng
- Để tính pred = U*I: tính từng vị trí và lưu vào một mảng

In [34]:
class Model():
  def __init__(self, data, theta, lamda, K, n_iter):
    self.data = data
    self.theta = theta
    self.lamda = lamda
    self.K = K
    self.n_iter = n_iter
    self.seed = 0
  
    # self.I = np.random.randn(K,data.n_items)
    # self.U = np.random.randn(K,data.n_users)
    self.U = np.load('/content/gdrive/MyDrive/Đại học/Data/rs_cf/model_U.npy')
    self.I = np.load('/content/gdrive/MyDrive/Đại học/Data/rs_cf/model_I.npy')

    self.i_c_train, self.u_c_train = self.data.Ytrain.nonzero()
    self.i_c_test, self.u_c_test = self.data.Ytest.nonzero()

    self.RMSE = []
  #u là cột,i là dòng
  def pred_(self, u, i):
    return np.dot(self.I[:,i].T,self.U[:,u])
    

  def error_(self, u, i):
    return self.data.Ytrain.getrow(i).toarray()[0,u] - self.pred_(u,i)

  def sgd_(self,u,i):
    error = self.error_(u,i)
    
    self.I[:,i] += self.theta*(error*self.U[:,u] - self.lamda*self.I[:,i])
    self.U[:,u] += self.theta*(error*self.I[:,i] - self.lamda*self.U[:,u])


  def rmse_(self):
    all_error = 0
    for i, u in zip(self.i_c_test, self.u_c_test):
      all_error += self.error_(u,i)*self.error_(u,i)
    
    return math.sqrt(all_error/len(self.i_c_test))

  def save_model(self):
    np.save('/content/gdrive/MyDrive/Đại học/Data/rs_cf/model_U.npy',self.U)
    np.save('/content/gdrive/MyDrive/Đại học/Data/rs_cf/model_I.npy',self.I)

  def fit(self):
    for iter in tqdm(range(self.n_iter)): 
      i_c_shuffle, u_c_shuffle = shuffle(model.i_c_train, model.u_c_train, random_state=self.seed)
      for i,u in zip(i_c_shuffle, u_c_shuffle):
        self.sgd_(u,i)
      
      rmse = self.rmse_()
      self.RMSE.append(rmse)
      self.save_model()
      print('iteration:',iter, 'RMSE:',rmse)

  def recommend (self, user_id):
    w_item_id = [x for x,y in zip(self.i_c_train, self.u_c_train) if y == user_id]
    ids_ = []
    ratings_ = []

    for i in range(self.data.n_items):
      if i not in w_item_id:
        ids_.append(i)
        ratings_.append(self.pred_(user_id,i))

    rs_df = pd.DataFrame({'item_id': ids_, 'rating': ratings_}).sort_values(by = ['rating'],ascending = False).reset_index(drop = True)
    return rs_df
  

In [35]:
model = Model(dataset, theta=0.001, lamda=0.1, K=3, n_iter=40)

In [ ]:
model.fit()

  0%|          | 0/40 [00:00<?, ?it/s]

iteration: 0 RMSE: 0.1940251015568368
iteration: 1 RMSE: 0.19206668127832963
iteration: 2 RMSE: 0.19050894349063538
iteration: 3 RMSE: 0.18924218575273358
iteration: 4 RMSE: 0.18819547613117368
iteration: 5 RMSE: 0.18732593367395697
iteration: 6 RMSE: 0.18660665720709496
iteration: 7 RMSE: 0.1860198811448836
iteration: 8 RMSE: 0.18555329636632326
iteration: 9 RMSE: 0.18519805123646277
iteration: 10 RMSE: 0.1849476323777249
iteration: 11 RMSE: 0.18479721514395012
iteration: 12 RMSE: 0.18474327279995087
iteration: 13 RMSE: 0.1847833333539204
iteration: 14 RMSE: 0.1849158239033433
iteration: 15 RMSE: 0.18513996893202556
iteration: 16 RMSE: 0.1854557232463732
iteration: 17 RMSE: 0.18586372808160478
iteration: 18 RMSE: 0.18636528333107868
iteration: 19 RMSE: 0.18696233140036528
iteration: 20 RMSE: 0.1876574496833749
iteration: 21 RMSE: 0.18845384954664485
iteration: 22 RMSE: 0.18935538023818801
iteration: 23 RMSE: 0.19036653644902155
iteration: 24 RMSE: 0.19149246843056655
iteration: 25 RMS

In [36]:
rs = model.recommend(model.u_c_train[9])

In [37]:
rs['item_ids'] = le.inverse_transform(rs['item_id'])
rs = rs.drop(['item_id'],axis=1) 

In [42]:
print('Recommend cho người dùng',le1.inverse_transform([model.u_c_train[9]])[0],'là:')
rs

Recommend cho người dùng 13001237 là:


,rating,item_ids
0,2.584030,122346
1,2.442493,17987
2,2.352141,126563
3,2.344478,124098
4,2.341103,13397
...,...,...
3462,-1.950787,128727
3463,-1.973794,125371
3464,-1.975692,6171
3465,-2.036491,13816


In [ ]:
np.save('/content/gdrive/MyDrive/Đại học/Data/rs_cf/tested.npy', np.array([[1, 2, 3], [4, 5, 6]]))
np.save('/content/gdrive/MyDrive/Đại học/Data/rs_cf/tested.npy', np.array([[7, 8, 9]]))

In [ ]:
np.load('/content/gdrive/MyDrive/Đại học/Data/rs_cf/tested.npy')


array([[7, 8, 9]])

In [39]:
model.u_c_train[9]

242085